***The following is just a rough draft/incomplete solution to the problem. I don't know if I'll be able to complete on time or not becuase of University stuff. But here it is as of now. I put heavy emphasis on the preprocessing. A lot of the preprocessing is stuff I learnt from my college course "Digital Image Processing".***

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
        print(dirname)

# Any results you write to the current directory are saved as output.

**Importing the training CSV file**

In [ ]:
base_path = "../input/kuzushiji-recognition/"
train = pd.read_csv("../input/kuzushiji-recognition/train.csv")

In [ ]:
train.head(5)

Let's observe the labels of any one element

In [ ]:
print(train["labels"][0])

On observing we see that, each of the of labels gives us bounding boxes in the format - "class id", "X_c", "Y_c", "width", "height". 

Let's take a look at one of the training images. 

In [ ]:
import cv2
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
s = train["image_id"][0]
img_0_path = base_path + "train_images/" + s + ".jpg"
img_0 = cv2.imread(img_0_path)
img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2RGB)
plt.imshow(img_0)

We observe some issues, which have been stated in the problem statement also.

1. You can occasionally see through especially thin paper and read characters from the opposite side of the page. Those characters should also be ignored.
2. Also, the annotations which are in smaller font size, they need to be ignored.

The first problem can be tackled by using thresholding. Let's see what happens thresholding is applied.

In [ ]:
#preprocessing on a single image

plt.figure(figsize=(20,20))
img_0_orig = cv2.imread(img_0_path)
img_0_orig = cv2.cvtColor(img_0_orig, cv2.COLOR_BGR2RGB)
img_0 = cv2.imread(img_0_path)
img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(img_0,(3,3),0)
sharp_mask = np.subtract(img_0, blur)
img_0 = cv2.addWeighted(img_0,1, sharp_mask,10, 0)
ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel_1 = np.ones((5,5),np.uint8)
kernel_2 = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel_1)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_2)
mask = cv2.bitwise_not(closing)
mask = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)
img = cv2.add(img_0_orig,mask)
blur_1 = cv2.GaussianBlur(img, (13,13), 0)
sharp_mask_1 = np.subtract(img,blur_1)
sharp_mask_1 = cv2.GaussianBlur(sharp_mask_1, (7,7), 0)
img = cv2.addWeighted(img,1,sharp_mask_1,-10, 0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(img_0_orig.shape)

So before putting any image we need to preprocess our image so that the print on the back page is not passed when the model is predicting.

Intuitively, I can see that the intensity values for that are higher than the Kuzushiji print on a page so, by using Otsu's Thresholding and I do binary thresholding of the images and I get the above result. The binary thresholding involves passing the image through a (3,3) Gaussian filter and then Otsu's thresholding is applied. On observing, we were right and we can no more see any from the opposite side of the page. To further improve the quality of the input, the image is passed through 2 morphological transformations, i.e. opening using a (5,5) one's kernel and a (1,1) one's closing kernel.

**End of all preprocessing**

In [ ]:
pwd = os.getcwd()
print(pwd)

**Making a list of all training images**

In [ ]:
train_imgs_path = base_path + "train_images/"
filelist = os.listdir(train_imgs_path)
#print(filelist[:5])


In [ ]:
trainlist = train["image_id"].tolist()
trainlist = [x + ".jpg" for x in trainlist]

In [ ]:
#print(trainlist[:5])

In [ ]:
train_out = train["labels"].tolist()
#train_out[:5]


In [ ]:
yolo_labels = []
img_label = []
for l in train_out:
    voc_out = str(l).split()
    for i in range(len(voc_out)//5):
        start_idx = 5*i
        img_label.append(voc_out[start_idx:start_idx+5])
    yolo_labels.append(img_label)
    img_label = []
#print(yolo_labels[:5])

In [ ]:
trainlist[0]

In [ ]:
def pre(Image):
    img_0_orig = cv2.imread(Image)
    img_0_orig = cv2.cvtColor(img_0_orig, cv2.COLOR_BGR2RGB)
    img_0 = cv2.imread(Image)
    img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img_0,(3,3),0)
    sharp_mask = np.subtract(img_0, blur)
    img_0 = cv2.addWeighted(img_0,1, sharp_mask,10, 0)
    ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel_1 = np.ones((5,5),np.uint8)
    kernel_2 = np.ones((1,1),np.uint8)
    opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel_1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_2)
    mask = cv2.bitwise_not(closing)
    mask = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)
    img = cv2.add(img_0_orig,mask)
    blur_1 = cv2.GaussianBlur(img, (13,13), 0)
    sharp_mask_1 = np.subtract(img,blur_1)
    sharp_mask_1 = cv2.GaussianBlur(sharp_mask_1, (7,7), 0)
    img = cv2.addWeighted(img,1,sharp_mask_1,-10, 0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
    

In [ ]:
plt.imshow(pre(train_imgs_path+trainlist[0]), cmap = 'gray')
print(pre(train_imgs_path+trainlist[0]).shape)

In [ ]:
#yolo_model = make_yolov3_model()
#yolo_model.load_weights("yolo.h5")

In [ ]:
#yolo_model.summary()

In [ ]:
uni_lab = pd.read_csv("../input/kuzushiji-recognition/unicode_translation.csv")

In [ ]:
#uni_lab.head()

In [ ]:
#uni_lab.tail()

In [ ]:
uni_list = uni_lab["Unicode"].to_list()
#print(uni_list[:5])

**Saving all of these in a class.names file**

In [ ]:
with open('class.names', 'w') as f:
    for item in uni_list:
        f.writelines(item+"\n")
#!ls

In [ ]:
#open('class.names', 'w').close()
with open("class.names", "r") as f_r:
    test_list = f_r.readlines()
#print(test_list[0][:-1])

**Creating a dictionary to map class_id to class_name**

In [ ]:
uni_dict = ((uni_list[i],i) for i in range(len(uni_list)))
uni_dict = dict(uni_dict)        

In [ ]:
#uni_dict["U+0031"]

**Changing the classnames to class ids in the yolo_labels list including the groundtruths**

In [ ]:
for labels in yolo_labels:
    for label in labels:
        label[0] = uni_dict[label[0]]
        label[1:5] = list(map(int, label[1:5]))
yolo_labels[:5]

*Converting the annotations into VOC format*

In [ ]:
for i in range(len(trainlist)):
    labels = yolo_labels[i]
    img = trainlist[i]
    img_path = base_path + "train_images/" + img
    image = Image.open(img_path)
    w, h = image.size
    for label in labels:
        id_1 = label[0]
        label[0],label[2] = list(map(int,[label[1]-label[3]/2,label[1]+label[3]/2]))
        label[1],label[3] = list(map(int,[label[2]-label[4]/2,label[2]+label[4]/2]))
        label[4] = id_1

In [ ]:
#yolo_labels[:3]

In [ ]:
#str(yolo_labels[0][0][0])

**Creating a dictionary which maps image ids to bounding boxes/detections**

In [ ]:
lines = [train_imgs_path+img_name for img_name in trainlist]
#print(lines[:3])
annot = []
for labels in yolo_labels:
    ann_fin = ""
    for label in labels:
        ann = " " + str(label[0])+','+str(label[1])+','+str(label[2])+','+str(label[3])+','+str(label[4])
        ann_fin += ann
    annot.append(ann_fin)
#print(annot[0])
for i in range(len(lines)):
    lines[i] = lines[i]+annot[i]
#print(lines[0])

This is all I could do before the end of the competition. Hope this helps someone.